In [ ]:
import torch
import numpy as np
import pandas as pd
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from torch import nn, optim
import torch.nn.functional as F


%matplotlib inline
%config InlineBackend.figure_format='retina'
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
rcParams['figure.figsize'] = 12, 8
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [ ]:
#1. load data
df = pd.read_csv('processed_segments_no_overlap.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df
# 0 for normal, 1 for abnormal


,curb_scene,Acc-Z_1,Acc-Z_2,Acc-Z_3,Acc-Z_4,Acc-Z_5,Acc-Z_6,Acc-Z_7,Acc-Z_8,Acc-Z_9,...,Acc-Z_91,Acc-Z_92,Acc-Z_93,Acc-Z_94,Acc-Z_95,Acc-Z_96,Acc-Z_97,Acc-Z_98,Acc-Z_99,Acc-Z_100
0,0.0,8.762772,8.753204,8.762772,8.781937,8.753204,8.748413,8.738831,8.748413,8.743622,...,8.786728,8.801086,8.820236,8.786728,8.777145,8.772354,8.781937,8.801086,8.781937,8.753204
1,0.0,8.781937,8.796295,8.772354,8.772354,8.772354,8.786728,8.781937,8.801086,8.767563,...,8.777145,8.753204,8.767563,8.762772,8.762772,8.753204,8.748413,8.762772,8.772354,8.767563
2,0.0,8.757996,8.762772,8.777145,8.767563,8.786728,8.777145,8.753204,8.757996,8.777145,...,8.757996,8.757996,8.772354,8.767563,8.757996,8.743622,8.777145,8.743622,8.786728,8.796295
3,0.0,8.757996,8.786728,8.791504,8.781937,8.772354,8.762772,8.781937,8.762772,8.767563,...,8.743622,8.762772,8.767563,8.757996,8.762772,8.781937,8.772354,8.767563,8.767563,8.801086
4,0.0,8.772354,8.762772,8.786728,8.767563,8.753204,8.762772,8.757996,8.757996,8.762772,...,8.777145,8.748413,8.791504,8.748413,8.767563,8.772354,8.772354,8.777145,8.767563,8.781937
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5486,1.0,1.781281,7.517792,7.517792,11.506531,18.310852,14.408310,5.880158,9.155426,6.181824,...,-9.974243,-4.079727,12.971787,35.094208,35.094208,29.678528,12.061981,3.869034,0.282516,1.613693
5487,1.0,6.866577,11.966217,13.469772,7.694961,4.419693,15.255859,23.113617,12.947845,12.947845,...,9.155426,15.145721,13.752289,1.455673,6.268021,19.474442,17.611755,17.611755,12.421112,3.189072
5488,1.0,-2.624039,-2.992752,12.392395,14.432251,24.406494,22.993912,10.745178,-1.216248,-4.831497,...,12.014099,10.970230,3.050217,9.270355,9.270355,16.275787,6.411667,6.411667,8.188171,5.961563
5489,1.0,0.009583,9.605545,12.990936,20.561401,15.883133,11.774689,13.302185,10.544067,17.769760,...,60.429657,60.429657,25.072083,9.495407,9.495407,-14.068329,-19.512741,-11.065994,2.049438,10.376465


In [15]:
#standardize data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data_scaled = scaler.fit_transform(df.drop('curb_scene', axis=1))
# Convert the scaled data back to DataFrames for further processing
data_scaled_df = pd.DataFrame(data_scaled, columns=df.columns[1:])
# Add the 'curb_scene' column back to the scaled DataFrames
data_scaled_df['curb_scene'] = df['curb_scene'].values
data_scaled_df.head()
# # save it in csv file
# #data_scaled_df.to_csv('processed_segments_no_overlap_scaled.csv')

,Acc-Z_1,Acc-Z_2,Acc-Z_3,Acc-Z_4,Acc-Z_5,Acc-Z_6,Acc-Z_7,Acc-Z_8,Acc-Z_9,Acc-Z_10,...,Acc-Z_92,Acc-Z_93,Acc-Z_94,Acc-Z_95,Acc-Z_96,Acc-Z_97,Acc-Z_98,Acc-Z_99,Acc-Z_100,curb_scene
0,-0.021925,-0.030469,-0.025929,-0.042453,-0.038768,-0.038232,-0.018914,-0.008354,-0.014091,-0.029113,...,-0.010031,-0.006064,-0.034178,-0.041356,-0.033399,-0.029454,-0.016349,-0.031946,-0.046974,0.0
1,-0.018551,-0.023017,-0.024217,-0.044124,-0.035523,-0.031557,-0.011542,-0.000052,-0.010149,-0.025919,...,-0.019075,-0.016196,-0.038654,-0.043962,-0.036873,-0.035171,-0.022756,-0.033648,-0.044372,0.0
2,-0.022766,-0.028814,-0.023361,-0.044960,-0.033087,-0.033227,-0.016456,-0.006843,-0.008571,-0.028314,...,-0.018170,-0.015275,-0.037759,-0.044828,-0.038612,-0.030271,-0.025958,-0.031095,-0.039165,0.0
3,-0.022766,-0.024671,-0.020795,-0.042453,-0.035523,-0.035731,-0.011542,-0.006091,-0.010149,-0.022726,...,-0.017268,-0.016196,-0.039547,-0.043962,-0.031661,-0.031088,-0.021955,-0.034499,-0.038297,0.0
4,-0.020238,-0.028814,-0.021649,-0.044960,-0.038768,-0.035731,-0.015636,-0.006843,-0.010937,-0.026718,...,-0.019980,-0.011591,-0.041337,-0.043093,-0.033399,-0.031088,-0.020353,-0.034499,-0.041767,0.0


In [24]:
#2. spilt data into normal and abnormal, 0 for normal, 1 for abnormal
normal_scaled_df = data_scaled_df[data_scaled_df['curb_scene'] == 0.0].drop('curb_scene', axis=1)
abnormal_scaled_df = data_scaled_df[data_scaled_df['curb_scene'] == 1.0].drop('curb_scene', axis=1)
print(normal_scaled_df.shape)
print(abnormal_scaled_df.shape)

(5304, 100)
(187, 100)


In [25]:
#3. split the normal examples into train, validation and test sets:

normal_train_scaled_df, normal_val_scaled_df = train_test_split(
    normal_scaled_df, 
    test_size=0.15, 
    random_state=RANDOM_SEED
)

normal_val_scaled_df, normal_test_scaled_df = train_test_split(
    normal_val_scaled_df,
    test_size=0.33,
    random_state=RANDOM_SEED
)

# Print the shapes to verify the splits
print("Training set shape:", normal_train_scaled_df.shape)
print("Validation set shape:", normal_val_scaled_df.shape)
print("Test set shape:", normal_test_scaled_df.shape)


Training set shape: (4508, 100)
Validation set shape: (533, 100)
Test set shape: (263, 100)


In [ ]:
#4. convert our examples into tensors, so we can use them to train our LSTM Autoencoder.